In [2]:
%load_ext autoreload
%autoreload 2

import sys
repo_dir = '/home/labs/amit/noamsh/repos/MM_2023'
sys.path.append(repo_dir)

In [4]:
from pathlib import Path
from datetime import date

import pandas as pd
import anndata as ad

## validate snapshot is equal to source

In [26]:
data_v = "2024-06-19"

In [21]:
data_dir = Path('/home/labs/amit/noamsh/data/mm_2023')

metadata_file_path = Path(data_dir, 'Blueprint_MM_Plates.xlsx')
metadata_snapshot_path = Path(data_dir, f'Blueprint_MM_Plates_{data_v}.xlsx')

In [22]:
for sheet in ["MM3_MARS", "MM3_SPID", "Blood_MARS", "Blood_SPID"]:
    orig_metadata = pd.read_excel(metadata_file_path, sheet_name=sheet)
    snap_metadata = pd.read_excel(metadata_snapshot_path, sheet_name=sheet)
    pd.testing.assert_frame_equal(snap_metadata, orig_metadata)

## split to csv, add Method columns

In [ ]:
sheets_name_map = {
    "MM3_MARS": "BM_MARS",
    "MM3_SPID": "BM_SPID",
    "Blood_MARS": "Blood_MARS",
    "Blood_SPID": "Blood_SPID"
}

new_plates_csv_paths = []

for sheet, new_name in sheets_name_map.items():
    snap_metadata = pd.read_excel(metadata_snapshot_path, sheet_name=sheet)
    snap_metadata["Method"] = "MARS" if "MARS" in sheet else "SPID"
    csv_name = f"Blueprint_MM_{new_name}_Plates_{data_v}.csv"
    csv_path = Path(data_dir, csv_name)
    
    new_plates_csv_paths.append(csv_path)
    snap_metadata.to_csv(csv_path)

### update metadata of .h5ad by current config

In [1]:
## load adata
adata_path = 
adata = ad.read(adata_path)

In [ ]:
## load current plate-level metadate
data_v = "2024-06-19"
data_dir = Path('/home/labs/amit/noamsh/data/mm_2023')
metadata_snapshot_path = Path(data_dir, f'Blueprint_MM_Plates_{data_v}.xlsx')

new_plates_data = pd.concat([pd.read_csv(df_path) for df_path in new_plates_csv_paths])

In [5]:
def update_obs_by_columns(adata: ad.AnnData, metadata_df: pd.DataFrame, update_by:str):
    df = metadata_df.copy()
    df = df.set_index(update_by)
    columns_to_update = df.columns
    
    adata.obs = adata.obs.drop(columns=[columns_to_update])
    adata.obs = adata.obs.merge(df, how="left", validate="m:1", left_on=update_by, right_index=True)

In [ ]:
adata_updated = update_obs_by_columns(adata, new_plates_data, "Amp.Batch")